In [3]:
using CSV, DataFrames, JuMP, Gurobi, Formatting, NPZ

# Loading Data

In [4]:
length_s2d_df = CSV.read("csv/l2_s2d.csv", DataFrame, header=false);
length_d2d_df = CSV.read("csv/l2_d2d.csv", DataFrame, header=false);

In [5]:
first(length_d2d_df, 2)

Row,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,4.8211,3.04137,2.33759,1.2254,1.96559,2.38026,2.2427,2.01504
2,4.8211,0.0,1.79856,3.02076,3.81632,2.87258,2.45442,2.59646,2.82419


In [6]:
n_src = nrow(length_s2d_df);
n_dst = ncol(length_s2d_df);

all_src = 1:n_src;
all_dst = 1:n_dst;

println("all_src: $all_src, all_dst: $all_dst")

all_src: 1:3, all_dst: 1:9


In [7]:
L_s2d = Matrix(length_s2d_df);
L_d2d = Matrix(length_d2d_df);

println("L_s2d: $(size(L_s2d)), L_d2d: $(size(L_d2d))")

L_s2d: (3, 9), L_d2d: (9, 9)


# Problem
- We now additionally consider that each robot has a fixed starting source.
- The number of robots is less than the number of destinations.

In [8]:
S = [2, 1, 1];
@assert size(S) == (n_src,)
@assert sum(S) < n_dst

println("There are a $n_dst destinations but only $(sum(S)) robots to visit them all.")

There are a 9 destinations but only 4 robots to visit them all.


In [9]:
# We assume that each robot visits a maximum of n_steps destinations.
n_steps = 3;
all_steps = 1:n_steps;
later_steps = 1:(n_steps-1);

println("n_steps: $n_steps, all_steps: $all_steps, later_steps: $later_steps")

n_steps: 3, all_steps: 1:3, later_steps: 1:2


# Minimum Distance

In [10]:
model2 = Model(Gurobi.Optimizer)

@variable(model2, x0[all_src, all_dst] >= 0, Int);
@variable(model2, x1[later_steps, all_dst, all_dst] >= 0, Int);

time_fromsrc = sum(sum(L_s2d[ii, jj] * x0[ii, jj] for ii in all_src) for jj in all_dst);
time_fromdst = sum(sum(sum(L_d2d[ii, jj] * x1[kk, ii, jj] for ii in all_dst) for jj in all_dst) for kk in later_steps);

@objective(model2, Min, time_fromsrc + time_fromdst);

# Each destination must be served by at least one source.
@constraint(model2, demand[jj in all_dst], sum(x0[ii, jj] for ii in all_src) + sum(sum(x1[kk, ii, jj] for ii in all_dst) for kk in later_steps) >= 1);

# Continuity constraints. x0_{ij} = x1_{jl}
@constraint(model2, cont0[jj in all_dst], sum(x0[ii, jj] for ii in all_src) == sum(x1[1, jj, ll] for ll in all_dst));
# Continuity constraints. x1_{ij} = x2_{jl}
@constraint(model2, cont1[kk in 1:(n_steps-2), jj in all_dst], sum(x1[kk, ii, jj] for ii in all_dst) == sum(x1[kk + 1, jj, ll] for ll in all_dst));

# Each robot must start from its assigned source.
@constraint(model2, start[ii in all_src], sum(x0[ii, jj] for jj in all_dst) == S[ii]);

# Solve.
optimize!(model2);
solution_summary(model2)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-19
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 9 3950X 16-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 30 rows, 189 columns and 486 nonzeros
Model fingerprint: 0xcc258200
Variable types: 0 continuous, 189 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 23.4622988
Presolve time: 0.00s
Presolved: 30 rows, 189 columns, 486 nonzeros
Variable types: 0 continuous, 189 integer (18 binary)
Found heuristic solution: objective 23.2983137

Root relaxation: objective 4.089825e+00, 28 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Une

* Solver : Gurobi

* Status
  Result count       : 9
  Termination status : OPTIMAL
  Message from the solver:
  "Model was solved to optimality (subject to tolerances), and an optimal solution is available."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 7.86256e+00
  Objective bound    : 7.86256e+00
  Relative gap       : 0.00000e+00
  Dual objective value : 7.86256e+00

* Work counters
  Solve time (sec)   : 1.08070e-02
  Barrier iterations : 0
  Node count         : 1


In [11]:
# Save solution.
x2_0 = Array(value.(x0));
x2_1 = Array(value.(x1));

npzwrite("sols/problem3_l2dist.npz", Dict("x0" => x2_0, "x1" => x2_1, "objective" => objective_value(model2)))

display(x2_0)
display(x2_1[1, :, :])
display(x2_1[2, :, :])

3×9 Matrix{Float64}:
  1.0  -0.0  -0.0   0.0   0.0   0.0  -0.0  -0.0   1.0
 -0.0   1.0   0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0   1.0  -0.0  -0.0  -0.0   0.0  -0.0  -0.0

9×9 Matrix{Float64}:
  1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0   1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0   1.0   0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0   0.0
 -0.0  -0.0  -0.0  -0.0  -0.0   0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0   0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0   0.0   0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0   0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0   1.0   0.0

9×9 Matrix{Float64}:
  0.0  -0.0  -0.0  -0.0   1.0  -0.0  -0.0  -0.0  -0.0
 -0.0   1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0   0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0   0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0   0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0   0.0   0.0  -0.0   0.0
 -0.0  -0.0  -0.0  -0.0  -0.0   1.0   0.0  -0.0  -0.0
 -0.0  -0.0  -0.0   1.0  -0.0  -0.0  -0.0   0.0   0.0
 -0.0  -0.0  -0.0   0.0  -0.0  -0.0  -0.0  -0.0  -0.0